# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
from impyute.imputation.cs import fast_knn
import plotly
import torch
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw
from scipy import stats

import warnings
warnings.filterwarnings("ignore", ".*will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.*")


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Update original data with new data from Mirny

## Load original data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/062_cringe_report"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

df_all = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
feats_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
ids_fimmu = df_all.index[(df_all['PMC10485620 ID'].notna()) & (df_all['Status'] == 'Control')].values
df = df_all.loc[ids_fimmu, :]

In [ ]:
age_window = 5
trgt_id = 'F2-L21'
trgt_age = df.at[trgt_id, 'Age']
trgt_simage = df.at[trgt_id, 'SImAge']
trgt_simage_acc = df.at[trgt_id, 'SImAge acceleration']
ids_near = df.index[(df['Age'] >= trgt_age - age_window) & (df['Age'] < trgt_age + age_window)]
trgt_simage_acc_prctl = stats.percentileofscore(df.loc[ids_near, 'SImAge acceleration'], trgt_simage_acc)

In [ ]:
sns.set(style='whitegrid', font_scale=1.5)
fig, ax = plt.subplots(figsize=(10, 6))
kdeplot = sns.kdeplot(
    data=df.loc[ids_near, :],
    x='SImAge acceleration',
    color='gray',
    linewidth=4,
    cut=0,
    ax=ax
)
kdeline = ax.lines[0]
xs = kdeline.get_xdata()
ys = kdeline.get_ydata()
ax.fill_between(xs, 0, ys, where=(xs <= trgt_simage_acc), interpolate=True, facecolor='dodgerblue', alpha=0.7)
ax.fill_between(xs, 0, ys, where=(xs >= trgt_simage_acc), interpolate=True, facecolor='crimson', alpha=0.7)
ax.vlines(trgt_simage_acc, 0, np.interp(trgt_simage_acc, xs, ys), color='black', linewidth=6)
ax.text(np.mean([min(xs), trgt_simage_acc]), 0.1 * max(ys), f"{trgt_simage_acc_prctl:0.1f}%", fontstyle = "oblique", color = "black", ha = "center", va = "center")
ax.text(np.mean([max(xs), trgt_simage_acc]), 0.1 * max(ys), f"{100- trgt_simage_acc_prctl:0.1f}%", fontstyle = "oblique", color = "black", ha = "center", va = "center")
fig.savefig(f"{path_save}/kde_aa.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/kde_aa.pdf", bbox_inches='tight')
plt.close(fig)

sns.set(style='whitegrid', font_scale=0.7)
n_rows = 2
n_cols = 5
fig_height = 4
fig_width = 10
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=False, sharex=False)
for feat_id, feat in tqdm(enumerate(feats_fimmu)):
    row_id, col_id = divmod(feat_id, n_cols)

    kdeplot = sns.kdeplot(
        data=df.loc[ids_near, :],
        x=feat,
        color='gray',
        linewidth=1,
        cut=0,
        ax=axs[row_id, col_id]
    )
    kdeline = axs[row_id, col_id].lines[0]
    xs = kdeline.get_xdata()
    ys = kdeline.get_ydata()
    trgt_val = df.at[trgt_id, feat]
    trgt_prctl = stats.percentileofscore(df.loc[ids_near, feat], trgt_val)
    axs[row_id, col_id].fill_between(xs, 0, ys, where=(xs <= trgt_val), interpolate=True, facecolor='dodgerblue', alpha=0.7)
    axs[row_id, col_id].fill_between(xs, 0, ys, where=(xs >= trgt_val), interpolate=True, facecolor='crimson', alpha=0.7)
    axs[row_id, col_id].vlines(trgt_val, 0, np.interp(trgt_val, xs, ys), color='black', linewidth=1.5)
    axs[row_id, col_id].text(np.mean([min(xs), trgt_val]), 0.1 * max(ys), f"{trgt_prctl:0.1f}%", fontstyle="oblique",
            color="black", ha="center", va="center")
    axs[row_id, col_id].text(np.mean([max(xs), trgt_val]), 0.1 * max(ys), f"{100 - trgt_prctl:0.1f}%", fontstyle="oblique",
            color="black", ha="center", va="center")
    axs[row_id, col_id].ticklabel_format(style='scientific', scilimits=(-1, 1), axis='y', useOffset=True)
fig.tight_layout()    
fig.savefig(f"{path_save}/kde_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/kde_feats.pdf", bbox_inches='tight')
plt.close(fig)
